# First MSSP-PD(NQP) example

In [2]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the synthetic instance of Section 3.2

In [3]:
nodes_number = 25
agents_number = 4
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               header=0,
                               names=["i", "j", "d_ij"])

# convert d_ij column into float
# print(synthetic_5x5_df.dtypes)
synthetic_5x5_df["d_ij"] = synthetic_5x5_df["d_ij"].str.replace(
    ",", ".").astype(np.float64)
# print(synthetic_5x5_df.dtypes)

# counting network nodes starting from 0
synthetic_5x5_df[["i", "j"]] = synthetic_5x5_df[["i", "j"]] - 1


Set up datas that will be used to solve the problem

In [4]:
Arc = namedtuple("Arc", synthetic_5x5_df.columns)
arcs = [Arc(*row) for row in synthetic_5x5_df.itertuples(index=False)]  # as List to avoid nusty bugs

nodes = np.unique(np.concatenate((synthetic_5x5_df["i"].to_numpy(),
                              synthetic_5x5_df["j"].to_numpy())))
agent_sources = np.array([0, 2, 3, 4])
agent_terminus = np.array([20, 22, 23, 24])

agents = range(agents_number)
# D = synthetic_5x5_df["d_ij"].to_numpy()

In [5]:
l = [k for k in agents for k_ in agents if k_<k]

print(l)


[1, 2, 2, 3, 3, 3]


## Manage the problem

Create the problem

In [4]:
MSPP_PD_NQP_pb = gb.Model("First MSPP_PD_NQP_pb")
MSPP_PD_NQP_pb.setParam("OutputFlag", 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


Define decision variables

In [ ]:
X_var_shape = len(nodes), len(nodes), agents_number

R_var_shape = len(nodes), agents_number

W_var_shape = len(nodes), agents_number, agents_number


X = MSPP_PD_NQP_pb.addMVar(X_var_shape,
                           vtype=GRB.BINARY,  # 5) Binary constraints
                           name="k-th agent traverse arc (i,j)")

R = MSPP_PD_NQP_pb.addMVar(R_var_shape,
                           vtype=GRB.BINARY,  # 13) Binary constraints
                           name="more agents traverse node i")

W = MSPP_PD_NQP_pb.addMVar(W_var_shape,
                           vtype=GRB.BINARY,  # 33) Non-negativity constraints
                           name="agents k and k_ both traverse node i")


Define the objective function

In [ ]:
# 1-3, 28) Objective function (linearized version)
distance_objn = gb.quicksum(
    arc.d_ij*X[arc.i, arc.j, k]
        for arc in arcs for k in agents
)
penalties_objn = gb.quicksum(
    W[i, k, k_] for i in nodes for k in agents for k_ in agents if k_<k
)


MSPP_PD_NQP_pb.setObjectiveN(distance_objn, index=0, weight=0.5,
name="Distance")
MSPP_PD_NQP_pb.setObjectiveN(penalties_objn, index=1, weight=0.5,
name="Penalty")


MSPP_PD_NQP_pb.ModelSense = GRB.MINIMIZE

Add constraints

In [ ]:
# 4) Flow constraints

def compute_flow(X, node, arcs, agent):
    flow_out = gb.quicksum(X[node, arc.j, agent] for arc in arcs if arc.i==node)
    flow_in = gb.quicksum(X[arc.i, node, agent] for arc in arcs if arc.j==node)
    return flow_out - flow_in


for k in agents:
    for i in nodes:
        if i == agent_sources[k]:
            MSPP_PD_NQP_pb.addConstr(compute_flow(X, i, arcs, k) == 1,
            name=f"Flow constr related to agent {k} in node {i}")
        elif i == agent_terminus[k]:
            MSPP_PD_NQP_pb.addConstr(compute_flow(X, i, arcs, k) == -1,
            name=f"Flow constr related to agent {k} in node {i}")
        else:
            MSPP_PD_NQP_pb.addConstr( compute_flow(X, i, arcs, k) == 0,
            name=f"Flow constr related to agent {k} in node {i}")
            

In [ ]:
# 10,11) Turning on r_i constraints

for k in agents:
    for node in nodes:
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] >= gb.quicksum(X[node, arc.j, k] for arc in arcs if arc.i==node)
        )
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] >= gb.quicksum(X[arc.i, node, k] for arc in arcs if arc.j==node)
        )

In [ ]:
# 29) Turning off r_i constraints

for k in agents:
    for node in nodes:
        MSPP_PD_NQP_pb.addConstr(
            R[node, k] <= (
                gb.quicksum(X[node, arc.j, k] for arc in arcs if arc.i==node) +
                gb.quicksum(X[arc.i, node, k] for arc in arcs if arc.j==node)
                )
        )

In [ ]:
# 30-32) Well-defined W variable

for node in nodes:
    for k in agents:
        for k_ in agents:
            if k_<k:
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, k_] <= R[node, k]
                )
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, K_] <= R[node, k_]
                )
                MSPP_PD_NQP_pb.addConstr(
                    W[node, k, k_] >= R[node, k] + R[node, k_] - 1
                )

Solve the problem

In [ ]:
MSPP_PD_NQP_pb.optimize()

Report results

In [ ]:
print("Result of the optimization is:")
if MSPP_PD_NQP_pb.Status == 2:
    print("optimal")
elif MSPP_PD_NQP_pb.Status == 3:
    print("infeasible")
elif MSPP_PD_NQP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal


In [ ]:
n_objectives = MSPP_PD_NQP_pb.NumObj
n_solutions = MSPP_PD_NQP_pb.SolCount

print (f"The optimization founds {n_solutions} solutions:")
# Notice that are not all optimal and are ranked from best to worst.
for sol in range(n_solutions):
    MSPP_PD_NQP_pb.params.SolutionNumber = sol

    print(f"Solution {sol}:", end="")
    for obj in range(n_objectives):
        MSPP_PD_NQP_pb.params.ObjNumber = obj
        print(f" {MSPP_PD_NQP_pb.ObjNName}={MSPP_PD_NQP_pb.ObjNVal}", end="")

    print()

    for k in agents:
        print(f"Agent {k} will follow the path:")
        for arc in arcs:
            if math.isclose(X.X[arc.i, arc.j, k], 1):
                print(f"{arc.i}->{arc.j}", end="\t")
        print("\n")

    print()

The optimization founds 3 solutions:
Solution 0: Distance=16.0 Penalty=1.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


Solution 1: Distance=18.6 Penalty=0.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


Solution 2: Distance=24.0 Penalty=0.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


